In [1]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from pyngrok import ngrok
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_directory = r'G:\Meu Drive\reconhecimento_caes\dados_organizados'

In [3]:
train_set = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels='inferred',
    validation_split = 0.2,
    subset = "training",
    seed=123,
    image_size = (224, 224),
    batch_size = 32,

)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels='inferred',
    validation_split = 0.2,
    subset = "validation",
    seed=123,
    image_size = (224, 224),
    batch_size = 32
)

Found 115 files belonging to 25 classes.
Using 92 files for training.
Found 115 files belonging to 25 classes.
Using 23 files for validation.


In [4]:
def preprocessing_pipeline(image: tf.Tensor, label: tf.Tensor):
  resized_image = tf.image.resize(image, [224, 224])
  normalized_image = tf.keras.applications.resnet50.preprocess_input(resized_image)

  return normalized_image, label

In [5]:
train_dataset = train_set.map(preprocessing_pipeline)
validation_dataset = validation_set.map(preprocessing_pipeline)

In [6]:
base_model = tf.keras.applications.ResNet50(
    include_top = False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

base_model.trainable = False

In [7]:
data_augmentation_layer = Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2),
], name="data_augmentation_layer")

In [8]:
model_aug = Sequential([
    data_augmentation_layer,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(units=25, activation='softmax')
])

In [10]:
model_aug.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [11]:
mlflow.set_experiment("Artigo - Reconhecimento de Cães")
mlflow.tensorflow.autolog()

with mlflow.start_run(run_name="Baseline Resnet50 com Augmentation - v1"):
    history = model_aug.fit(
        train_dataset,
        epochs=10,
        validation_data=validation_dataset
    )

2025/07/14 21:50:32 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential' has no defined input shape yet.


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0494 - loss: 4.1320

3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.0506 - loss: 4.1009 - val_accuracy: 0.0435 - val_loss: 3.3959
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0494 - loss: 3.3959

3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.0506 - loss: 3.3614 - val_accuracy: 0.0870 - val_loss: 3.2721
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2328 - loss: 2.4374

3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.2398 - loss: 2.4319 - val_accuracy: 0.1739 - val_loss: 2.8363
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4330 - loss: 1.9219 

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 0.8886 - loss: 0.7961 - val_accuracy: 0.6087 - val_loss: 1.4992
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8102 - loss: 0.7094

3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.8169 - loss: 0.7037 - val_accuracy: 0.6522 - val_loss: 1.3780
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9189 - loss: 0.5485

3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9175 - loss: 0.5471 - val_accuracy: 0.6957 - val_loss: 1.3201
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9912 - loss: 0.3864

3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9907 - loss: 0.3763 - val_accuracy: 0.6522 - val_loss: 1.2758
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


2025/07/14 21:52:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [ ]:
print("Avaliando o modelo diretamente no conjunto de validação...")
results = model_aug.evaluate(validation_dataset)
print("\nResultados da Avaliação:")
print(f"Perda de Validação (Loss): {results[0]}")
print(f"Acurácia de Validação (Accuracy): {results[1]}")

Avaliando o modelo diretamente no conjunto de validação...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6957 - loss: 1.3411

Resultados da Avaliação:
Perda de Validação (Loss): 1.3410615921020508
Acurácia de Validação (Accuracy): 0.695652186870575


In [12]:
mlflow.set_experiment("Artigo - Reconhecimento de Cães")
mlflow.tensorflow.autolog()

with mlflow.start_run(run_name="Baseline Resnet50 com Augmentation - v2 - 30 Epocas"):
    history = model_aug.fit(
        train_dataset,
        epochs=30,
        initial_epoch=10, 
        validation_data=validation_dataset    
    )

Epoch 11/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9205 - loss: 0.3771

3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 0.9159 - loss: 0.3804 - val_accuracy: 0.6957 - val_loss: 1.2278
Epoch 12/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9490 - loss: 0.3365

3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.9509 - loss: 0.3280 - val_accuracy: 0.6957 - val_loss: 1.1924
Epoch 13/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9771 - loss: 0.2075

3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9774 - loss: 0.2058 - val_accuracy: 0.6957 - val_loss: 1.1435
Epoch 14/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9808 - loss: 0.1694

3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9828 - loss: 0.1653 - val_accuracy: 0.6957 - val_loss: 1.0963
Epoch 15/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9964 - loss: 0.1466

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.9907 - loss: 0.1557 - val_accuracy: 0.7391 - val_loss: 0.9696
Epoch 19/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - loss: 0.0820

3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 1.0000 - loss: 0.0830 - val_accuracy: 0.7391 - val_loss: 0.9642
Epoch 20/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 1.0000 - loss: 0.0800 

3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 5s/step - accuracy: 1.0000 - loss: 0.0816 - val_accuracy: 0.7826 - val_loss: 0.9544
Epoch 21/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - loss: 0.0905

3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 1.0000 - loss: 0.0924 - val_accuracy: 0.7826 - val_loss: 0.9462
Epoch 22/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.9946 - loss: 0.0889 - val_accuracy: 0.7826 - val_loss: 0.9470
Epoch 23/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 1.0000 - loss: 0.0866 - val_accuracy: 0.7826 - val_loss: 0.9561
Epoch 24/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 1.0000 - loss: 0.0613 - val_accuracy: 0.7826 - val_loss: 0.9674
Epoch 25/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 1.0000 - loss: 0.0617 - val_accuracy: 0.7826 - val_loss: 0.9805
Epoch 26/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 1.0000 - loss: 0.0630 - val_accuracy: 0.7826 - val_loss: 0.9881
Epoch 27/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 1.0000 - loss: 0.0665 - val_accuracy: 0.7391 - val_loss: 0.9865
Epoch 28/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 1.0000 - loss: 0.0482 - val_accuracy: 0.7391 - val_loss: 0.9877
Epoch 29/30


2025/07/14 22:01:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
